In [0]:
from google.colab import files

uploaded = files.upload()

In [56]:
import keras
import h5py
import numpy as np
from sklearn import metrics
from matplotlib import pyplot as plt 


from keras import backend as K
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D , Activation
from keras.models import model_from_json

#manually setting the image_data_format
K.set_image_data_format('channels_first')

plt.rcParams['figure.figsize'] = (8, 6)


# from skimage import io
# def show(img):
	# io.imshow(img)
	# io.show()
	
def softmax_c(z):
	assert len(z.shape) == 2
	s = np.max(z, axis=1)
	s = s[:, np.newaxis]
	e_x = np.exp(z - s)
	div = np.sum(e_x, axis=1)
	div = div[:, np.newaxis] 
	return e_x / div

def prepare_softtargets(model,X):
	inp = model.input                                           # input placeholder
	outputs = []
	for layer in model.layers[:]:
		if layer.name == 'flatten_1':
			outputs.append(layer.output)
		if layer.name == 'dense_2':
			outputs.append(layer.output)
			
	functor = K.function([inp]+ [K.learning_phase()], outputs ) # evaluation function
	layer_outs = functor([X, 1.])
	return np.array(layer_outs[0]) , np.array(layer_outs[1])


# Todo parse as cmd argments 
TRAIN_BIG = False  
TRAIN_SMALL = False 
PRPARE_TRAININPUT = True 
PRPARE_TESTINPUT = True 

# big model 
batch_size = 128
num_classes = 10
BIG_epochs = 20

# Small Model
STUDENT_epochs = 30
HiddenNeuron = 8   # found out after experimentation. 


### Setup Data 
#load cifar10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)



#load model architecture
with open(r'C10M.json', 'r') as f:
            model = model_from_json(f.read())
    
model.compile(loss='categorical_crossentropy',
			  optimizer=keras.optimizers.Adam(lr=0.0001, decay=1e-6),
			  metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 30, 30)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 30, 30)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 15, 15)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 15, 15)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 15, 15)        18496     
__________

In [0]:
#normalize the data
x_train = x_train  / 255.0
x_test = x_test / 255.0

In [58]:
print ("loading weights TRAIN_BIG FLAG set as FALSE")
model.load_weights(r'C10M.h5')

print ("Evaluating Initial Model ...\n")
score = model.evaluate(x_test , y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print ('-'*30)

print ("Parameter Size of Initial Model and Memory Footprint ")
trainable_params = model.count_params()
footprint = trainable_params * 4
print ("Memory footprint per Image Feed Forward ~= " , footprint / 1024.0 /1024.0 ,"Mb") # 2x Backprop
print ('-'*30)


## Obtaining the Output of the last Convolutional Layer After Flatten. Caruana et. al 
## and Preparing the SoftTargets, (Logits), as proposed Geoffery Hinton et. al 



if PRPARE_TRAININPUT:
	print ("Creating trainsfer Set")

	lastconv_out = []
	logit_out = []
	for i in range(0,50):
		print ("Batch # : ",i)
		l,l2 =  (prepare_softtargets(model,x_train[i*1000:(i+1)*1000]))
		lastconv_out.append(l)
		logit_out.append(l2)

	lastconv_out = np.array(lastconv_out)
	logit_out = np.array(logit_out)
	lastconv_out = lastconv_out.reshape((50000 , 2304))
	logit_out = logit_out.reshape((50000 , 10))

	print ("clean up ")
	x_train = 0
	print ("Write to Disk")
	h5f = h5py.File('new_lastconv_out.h5', 'w')
	h5f.create_dataset('dataset_1', data=lastconv_out)
	h5f.close()
	h5f2 = h5py.File('new_logit_out.h5', 'w')
	h5f2.create_dataset('dataset_1', data=logit_out)
	h5f2.close()

else:
	print ("loading Transfer Set from lastconv_out.h5")
	h5f = h5py.File('lastconv_out.h5' , 'r')
	lastconv_out = h5f['dataset_1'][:]
	h5f.close()

	h5f2 = h5py.File('logit_out.h5' , 'r')
	logit_out = h5f2['dataset_1'][:]
	h5f2.close()



print ("Building minimal Model")

student_model = Sequential()
student_model.add(Dense(HiddenNeuron,input_dim=2304,activation='relu'))
student_model.add(Dropout(0.2))
student_model.add(Dense(num_classes))

student_model.compile(loss='mse',
			  optimizer=keras.optimizers.Adadelta(),
			  metrics=['accuracy'])


TRAIN_SMALL = True

if TRAIN_SMALL: 
	print ("Training Small Model ")
	student_model.fit(lastconv_out,logit_out,nb_epoch=STUDENT_epochs,verbose=1 , batch_size=batch_size)
	student_model.save_weights("new_student_weights_6_0.2dopout.h5")
else :
	print ("Loading Small Model Weights")
	student_model.load_weights("student_weights_6_0.2dopout.h5")



print ("Clean up small model Training and targets")
lastconv_out = 0
logit_out = 0 


############ Preparing Test Input #########

if PRPARE_TESTINPUT:
	print ("creating Test data from the big Model on HeldOut data")
	
	test_lastconv_out = []
	test_logit_out = []
	for i in range(0,10):
		print ("Batch # : ",i)
		l,l2 =  prepare_softtargets(model,x_test[i*1000:(i+1)*1000])
		test_lastconv_out.append(l)
		test_logit_out.append(l2)    
	
	# lastconv_out.shape , logit_out.shape
	test_lastconv_out = np.array(test_lastconv_out)
	test_logit_out = np.array(test_logit_out)

	test_lastconv_out = test_lastconv_out.reshape((10000 , 2304))
	test_logit_out = test_logit_out.reshape((10000 , 10))

	print (test_lastconv_out.shape)
	print (test_logit_out.shape)

	print ("Write to Disk")
	h5f = h5py.File('new_test_lastconv_out.h5', 'w')
	h5f.create_dataset('dataset_1', data=lastconv_out)
	h5f.close()
	h5f2 = h5py.File('new_test_logit_out.h5', 'w')
	h5f2.create_dataset('dataset_1', data=logit_out)
	h5f2.close()
else :
	print ("Loading saved test data from .h5 . ")
	h5f = h5py.File('test_lastconv_out.h5' , 'r')
	test_lastconv_out = h5f['dataset_1'][:]
	h5f.close()
	h5f2 = h5py.File('test_logit_out.h5' , 'r')
	test_logit_out = h5f2['dataset_1'][:]
	h5f2.close()
	
pred = student_model.predict(test_lastconv_out)
probs = softmax_c(pred)
pred_classes = np.argmax(probs,axis=1)

accuracy_student = metrics.accuracy_score(y_pred=pred_classes,y_true=np.argmax(y_test,axis=1))
print ("Small Model Test Set Accuracy : " , accuracy_student)
print ("\n")
# Compression Rate from Number of Parameters Reduced

# Parameters for the first two conv layers from Bigger model
convparams = 320 + 18496

print ("Evaluating COompression ... ")
print ("HiddenNeurons : " , HiddenNeuron)
print ("Initial Model Parameters : " , model.count_params())
print ("Compressed Model parameters + initial feature extractor part params : ", student_model.count_params() + convparams)
compressionRate = model.count_params() / np.float(student_model.count_params()  + convparams)
print ("Compression Rate : " , compressionRate)
print ("\n\n")

# if __name__ == '__main__':
	# Set usage using flags. 
	# python runme.py --TRAIN_BIG=True --epochs=20 

loading weights TRAIN_BIG FLAG set as FALSE
Evaluating Initial Model ...

10000/10000 [==============================] - 2s 174us/step
Test loss: 0.6733700441360474
Test accuracy: 0.7846
------------------------------
Parameter Size of Initial Model and Memory Footprint 
Memory footprint per Image Feed Forward ~=  4.771644592285156 Mb
------------------------------
Creating trainsfer Set
Batch # :  0
Batch # :  1
Batch # :  2
Batch # :  3
Batch # :  4
Batch # :  5
Batch # :  6
Batch # :  7
Batch # :  8
Batch # :  9
Batch # :  10
Batch # :  11
Batch # :  12
Batch # :  13
Batch # :  14
Batch # :  15
Batch # :  16
Batch # :  17
Batch # :  18
Batch # :  19
Batch # :  20
Batch # :  21
Batch # :  22
Batch # :  23
Batch # :  24
Batch # :  25
Batch # :  26
Batch # :  27
Batch # :  28
Batch # :  29
Batch # :  30
Batch # :  31
Batch # :  32
Batch # :  33
Batch # :  34
Batch # :  35
Batch # :  36
Batch # :  37
Batch # :  38
Batch # :  39
Batch # :  40
Batch # :  41
Batch # :  42
Batch # :  43
Bat

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/30
50000/50000 [==============================] - 3s 61us/step - loss: 24.2106 - acc: 0.3700
Epoch 2/30
50000/50000 [==============================] - 2s 49us/step - loss: 18.6041 - acc: 0.4733
Epoch 3/30
50000/50000 [==============================] - 2s 50us/step - loss: 17.7811 - acc: 0.5040
Epoch 4/30
50000/50000 [==============================] - 3s 50us/step - loss: 17.3465 - acc: 0.5170
Epoch 5/30
50000/50000 [==============================] - 3s 50us/step - loss: 16.9858 - acc: 0.5287
Epoch 6/30
50000/50000 [==============================] - 2s 50us/step - loss: 16.9164 - acc: 0.5338
Epoch 7/30
50000/50000 [==============================] - 3s 50us/step - loss: 16.6951 - acc: 0.5408
Epoch 8/30
50000/50000 [==============================] - 2s 50us/step - loss: 16.6523 - acc: 0.5433
Epoch 9/30
50000/50000 [==============================] - 3s 50us/step - loss: 16.5705 - acc: 0.5416
Epoch 10/30
50000/50000 [==============================] - 2s 49us/step - loss: 16.4863 - a

In [0]:
pred = student_model.predict(lastconv_out)
probs = softmax_c(pred)
pred_classes = np.argmax(probs,axis=1)

accuracy_student = metrics.accuracy_score(y_pred=pred_classes,y_true=np.argmax(y_test,axis=1))
accuracy_student